In [1]:
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SequentialChain
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.callbacks import get_openai_callback
import PyPDF2

c:\Users\Administrator\Documents\mcqgen-gemini\env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
from langchain import PromptTemplate, LLMChain
import os
import getpass
import pandas as pd
from dotenv import load_dotenv, find_dotenv
import traceback
import json



In [6]:
load_dotenv(find_dotenv(), override=True)
if 'GOOGLE_API_KEY' not in os.environ:
    os.environ['GOOGLE_API_KEY'] = getpass.getpass('Provide your Google API Key: ')

In [8]:
# llm=ChatGoogleGenerativeAI(openai_api_key=KEY, model_name="gemini-1.5-flash", temperature=0.5)

In [11]:
llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash",
    temperature=0.5,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    # other params...
)

In [12]:
llm

ChatGoogleGenerativeAI(model='models/gemini-1.5-flash', google_api_key=SecretStr('**********'), temperature=0.5, max_retries=2, client=<google.ai.generativelanguage_v1beta.services.generative_service.client.GenerativeServiceClient object at 0x00000202225CBBB0>, async_client=<google.ai.generativelanguage_v1beta.services.generative_service.async_client.GenerativeServiceAsyncClient object at 0x00000202225F8D60>, default_metadata=())

In [14]:
RESPONSE_JSON = {
    "1": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "2": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "3": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
}

In [15]:
TEMPLATE="""
Text:{text}
You are an expert MCQ maker. Given the above text, it is your job to \
create a quiz  of {number} multiple choice questions for {subject} students in {tone} tone. 
Make sure the questions are not repeated and check all the questions to be conforming the text as well.
Make sure to format your response like  RESPONSE_JSON below  and use it as a guide. \
Ensure to make {number} MCQs
### RESPONSE_JSON
{response_json}

"""

In [16]:
quiz_generation_prompt = PromptTemplate(
    input_variables=["text", "number", "subject", "tone", "response_json",],
    template=TEMPLATE
)

In [17]:
quiz_chain=LLMChain(llm=llm, prompt=quiz_generation_prompt, output_key="quiz", verbose=True)

C:\Users\Administrator\AppData\Local\Temp\ipykernel_19076\2669661367.py:1: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  quiz_chain=LLMChain(llm=llm, prompt=quiz_generation_prompt, output_key="quiz", verbose=True)


In [18]:
TEMPLATE2="""
You are an expert english grammarian and writer. Given a Multiple Choice Quiz for {subject} students.\
You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for complexity analysis. 
if the quiz is not at per with the cognitive and analytical abilities of the students,\
update the quiz questions which needs to be changed and change the tone such that it perfectly fits the student abilities
Quiz_MCQs:
{quiz}

Check from an expert English Writer of the above quiz:
"""

In [19]:
quiz_evaluation_prompt=PromptTemplate(input_variables=["subject", "quiz"], template=TEMPLATE)


In [20]:
review_chain=LLMChain(llm=llm, prompt=quiz_evaluation_prompt, output_key="review", verbose=True)


In [21]:
generate_evaluate_chain=SequentialChain(chains=[quiz_chain, review_chain], input_variables=["text", "number", "subject", "tone", "response_json"],
                                        output_variables=["quiz", "review"], verbose=True,)

In [22]:
file_path= r"C:\Users\Administrator\Documents\mcqgen-gemini\data.txt"

In [23]:
with open(file_path, 'r') as file:
    TEXT =  file.read()

In [24]:
json.dumps(RESPONSE_JSON)

'{"1": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "2": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "3": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}}'

In [25]:
NUMBER=5 
SUBJECT="Machine Learning"
TONE="simple"

In [26]:
response=generate_evaluate_chain(
        {
            "text": TEXT,
            "number": NUMBER,
            "subject":SUBJECT,
            "tone": TONE,
            "response_json": json.dumps(RESPONSE_JSON)
        }
        )

C:\Users\Administrator\AppData\Local\Temp\ipykernel_19076\1882050128.py:1: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response=generate_evaluate_chain(
Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")
Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")


Prompt after formatting:

Text:Machine learning (ML) is a field of study in artificial intelligence concerned with the development and study of statistical algorithms that can learn from data and generalize to unseen data and thus perform tasks without explicit instructions.[1] Recently, artificial neural networks have been able to surpass many previous approaches in performance.[2]

ML finds application in many fields, including natural language processing, computer vision, speech recognition, email filtering, agriculture, and medicine.[3][4] When applied to business problems, it is known under the name predictive analytics. Although not all machine learning is statistically based, computational statistics is an important source of the field's methods.

The mathematical foundations of ML are provided by mathematical optimization (mathematical programming) methods. Data mining is a related (parallel) field of study, focusing on exploratory data analysis (EDA) through unsupervised learn

Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")



> Finished chain.
Prompt after formatting:

Text:Machine learning (ML) is a field of study in artificial intelligence concerned with the development and study of statistical algorithms that can learn from data and generalize to unseen data and thus perform tasks without explicit instructions.[1] Recently, artificial neural networks have been able to surpass many previous approaches in performance.[2]

ML finds application in many fields, including natural language processing, computer vision, speech recognition, email filtering, agriculture, and medicine.[3][4] When applied to business problems, it is known under the name predictive analytics. Although not all machine learning is statistically based, computational statistics is an important source of the field's methods.

The mathematical foundations of ML are provided by mathematical optimization (mathematical programming) methods. Data mining is a related (parallel) field of study, focusing on exploratory data analysis (EDA) through

In [27]:
response

{'text': 'Machine learning (ML) is a field of study in artificial intelligence concerned with the development and study of statistical algorithms that can learn from data and generalize to unseen data and thus perform tasks without explicit instructions.[1] Recently, artificial neural networks have been able to surpass many previous approaches in performance.[2]\n\nML finds application in many fields, including natural language processing, computer vision, speech recognition, email filtering, agriculture, and medicine.[3][4] When applied to business problems, it is known under the name predictive analytics. Although not all machine learning is statistically based, computational statistics is an important source of the field\'s methods.\n\nThe mathematical foundations of ML are provided by mathematical optimization (mathematical programming) methods. Data mining is a related (parallel) field of study, focusing on exploratory data analysis (EDA) through unsupervised learning.[6][7]\n\nFr

In [28]:
quiz=response.get("quiz")

In [29]:
quiz

'### RESPONSE_JSON\n```json\n{"1": {"mcq": "What is the primary goal of machine learning?", "options": {"a": "To create artificial intelligence that can think like humans", "b": "To develop algorithms that can learn from data and make predictions", "c": "To replace human jobs with automated systems", "d": "To understand the human brain and its cognitive processes"}, "correct": "b"}, "2": {"mcq": "Which of the following is NOT a common application of machine learning?", "options": {"a": "Natural language processing", "b": "Computer vision", "c": "Predictive analytics", "d": "Astrophysics"}, "correct": "d"}, "3": {"mcq": "What is the term used for machine learning when applied to business problems?", "options": {"a": "Data mining", "b": "Predictive analytics", "c": "Computational statistics", "d": "Artificial neural networks"}, "correct": "b"}, "4": {"mcq": "Who coined the term \'machine learning\'?", "options": {"a": "Alan Turing", "b": "Donald Hebb", "c": "Arthur Samuel", "d": "Tom M. 

In [30]:
quiz=json.loads(quiz)

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [31]:
quiz_table_data = []
for key, value in quiz.items():
    mcq = value["mcq"]
    options = " | ".join(
        [
            f"{option}: {option_value}"
            for option, option_value in value["options"].items()
            ]
        )
    correct = value["correct"]
    quiz_table_data.append({"MCQ": mcq, "Choices": options, "Correct": correct})

AttributeError: 'str' object has no attribute 'items'

In [32]:
print(quiz)

### RESPONSE_JSON
```json
{"1": {"mcq": "What is the primary goal of machine learning?", "options": {"a": "To create artificial intelligence that can think like humans", "b": "To develop algorithms that can learn from data and make predictions", "c": "To replace human jobs with automated systems", "d": "To understand the human brain and its cognitive processes"}, "correct": "b"}, "2": {"mcq": "Which of the following is NOT a common application of machine learning?", "options": {"a": "Natural language processing", "b": "Computer vision", "c": "Predictive analytics", "d": "Astrophysics"}, "correct": "d"}, "3": {"mcq": "What is the term used for machine learning when applied to business problems?", "options": {"a": "Data mining", "b": "Predictive analytics", "c": "Computational statistics", "d": "Artificial neural networks"}, "correct": "b"}, "4": {"mcq": "Who coined the term 'machine learning'?", "options": {"a": "Alan Turing", "b": "Donald Hebb", "c": "Arthur Samuel", "d": "Tom M. Mitch

In [33]:
print(quiz)


### RESPONSE_JSON
```json
{"1": {"mcq": "What is the primary goal of machine learning?", "options": {"a": "To create artificial intelligence that can think like humans", "b": "To develop algorithms that can learn from data and make predictions", "c": "To replace human jobs with automated systems", "d": "To understand the human brain and its cognitive processes"}, "correct": "b"}, "2": {"mcq": "Which of the following is NOT a common application of machine learning?", "options": {"a": "Natural language processing", "b": "Computer vision", "c": "Predictive analytics", "d": "Astrophysics"}, "correct": "d"}, "3": {"mcq": "What is the term used for machine learning when applied to business problems?", "options": {"a": "Data mining", "b": "Predictive analytics", "c": "Computational statistics", "d": "Artificial neural networks"}, "correct": "b"}, "4": {"mcq": "Who coined the term 'machine learning'?", "options": {"a": "Alan Turing", "b": "Donald Hebb", "c": "Arthur Samuel", "d": "Tom M. Mitch

In [34]:
quiz_data = json.loads(quiz)

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [35]:
if quiz:
    try:
        quiz_data = json.loads(quiz)
    except json.JSONDecodeError as e:
        print(f"Error decoding JSON: {e}")
else:
    print("No data to load as JSON")


Error decoding JSON: Expecting value: line 1 column 1 (char 0)


In [38]:
response

{'text': 'Machine learning (ML) is a field of study in artificial intelligence concerned with the development and study of statistical algorithms that can learn from data and generalize to unseen data and thus perform tasks without explicit instructions.[1] Recently, artificial neural networks have been able to surpass many previous approaches in performance.[2]\n\nML finds application in many fields, including natural language processing, computer vision, speech recognition, email filtering, agriculture, and medicine.[3][4] When applied to business problems, it is known under the name predictive analytics. Although not all machine learning is statistically based, computational statistics is an important source of the field\'s methods.\n\nThe mathematical foundations of ML are provided by mathematical optimization (mathematical programming) methods. Data mining is a related (parallel) field of study, focusing on exploratory data analysis (EDA) through unsupervised learning.[6][7]\n\nFr

In [42]:
quiz= response.get("quiz")

In [43]:
quiz=json.loads(quiz)

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [41]:
review_data = response.get("reviews")